# LAB 2

## BY Lucas Cheistwer

In [1]:
import nltk

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


## TAGSETS

In [6]:
nltk.help.upenn_tagset("NN*")

NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNPS: noun, proper, plural
    Americans Americas Amharas Amityvilles Amusements Anarcho-Syndicalists
    Andalusians Andes Andruses Angels Animals Anthony Antilles Antiques
    Apache Apaches Apocrypha ...
NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...


### Categorias de tipo Noun(Penn Treebank tagset):

#### NN
Sustantivos comunes en singular, que sirven para identificar 
en forma general a las personas, animales y objetos.

#### NNP 
Sustantivos propios en singular, aquellas que denominan en forma concreta
una entidad. Nombres, lugares, apodos.

#### NNPS
Son NNP pero en plural.

#### NNS
Son NN pero en plural.

##  Contando desde un corpora

### Cuál es el tamaño del Brown corpus?

In [15]:
from nltk.corpus import brown

In [16]:
len(brown.tagged_words())

1161192

#### Creo tupla de NN

In [20]:
race1 = nltk.tag.str2tuple('race/NN')

#### Creo tupla de VB

In [21]:
race2 = nltk.tag.str2tuple('race/VB')

### Que tan frecuente aparecen estas tuplas en el corpus?

In [22]:
cantNN = brown.tagged_words().count(race1)
cantNN

94

In [23]:
cantVB = brown.tagged_words().count(race2)
cantVB

4

Claramente su uso mas frecuente es como sustantivo. La palabra race como verbo aparece unicamente 4 veces. 

## POS TAGGING

### El POS tagger HMM que se detalla en las diapositivas de la clase, usa dos fuentes de información. Una es la probabilidad de una palabra dado un tag particular, p(wi|ti). Cuál es la otra fuente de información que te ayuda a tagear una oración?

La otra fuente de informacion es lo que se dice el Prior. Esta es la probabilidad
de una secuencia de n tags. No depende de la oracion, por lo tanto determina
cuan probable es una secuencia de tags en particular.

#### Seleccionamos solo los documentos de noticias de la colección diversa que tiene el corpus Brown.

In [27]:
unigram_tagger = nltk.tag.UnigramTagger(brown.tagged_sents(categories='news')[:5000])

#### Tenemos la frase: "The Secretariat is expected to race tomorrow"

Imagino que al no tener en cuenta el contexto, a la palabra race se le asignara el TAG NN, ya que es el mas probable.

Y, efectivamente:

In [28]:
from nltk import word_tokenize

In [29]:
S = "The Secretariat is expected to race tomorrow."
S_tok = word_tokenize(S)
unigram_tagger.tag(S_tok)

[('The', 'AT'),
 ('Secretariat', 'NN-TL'),
 ('is', 'BEZ'),
 ('expected', 'VBN'),
 ('to', 'TO'),
 ('race', 'NN'),
 ('tomorrow', 'NR'),
 ('.', '.')]

#### Ahora vamos a probar con el modelo HMM, que tiene en cuenta el contexto.

In [39]:
hmm_tagger = nltk.hmm.HiddenMarkovModelTrainer().train_supervised(brown.tagged_sents(categories="news")[:5000])
hmm_tagger.tag(S_tok)

[('The', 'AT'),
 ('Secretariat', 'NN-TL'),
 ('is', 'BEZ'),
 ('expected', 'VBN'),
 ('to', 'TO'),
 ('race', 'VB'),
 ('tomorrow', 'NR'),
 ('.', '.')]

### Probá los dos modelos Unigram y HMM para otras sentencias ambiguas que se te ocurran

#### Primer intento fallido:

In [41]:
S2 = "I would love to party next weekend"
S2_tok = word_tokenize(S2)
unigram_tagger.tag(S2_tok)

[('I', 'PPSS'),
 ('would', 'MD'),
 ('love', 'VB'),
 ('to', 'TO'),
 ('party', 'NN'),
 ('next', 'AP'),
 ('weekend', 'NN')]

In [42]:
hmm_tagger.tag(S2_tok)

[('I', 'PPSS'),
 ('would', 'MD'),
 ('love', 'VB'),
 ('to', 'IN'),
 ('party', 'NN'),
 ('next', 'AP'),
 ('weekend', 'NN')]

La palabra party es un verbo y el HMM no la tageó como tal.

#### El segundo intento funcionó:

In [43]:
S3 = "I will have to object to that decision"
S3_tok = word_tokenize(S3)
unigram_tagger.tag(S3_tok)

[('I', 'PPSS'),
 ('will', 'MD'),
 ('have', 'HV'),
 ('to', 'TO'),
 ('object', 'NN'),
 ('to', 'TO'),
 ('that', 'CS'),
 ('decision', 'NN')]

In [44]:
hmm_tagger.tag(S3_tok)

[('I', 'PPSS'),
 ('will', 'MD'),
 ('have', 'HV'),
 ('to', 'TO'),
 ('object', 'VB'),
 ('to', 'IN'),
 ('that', 'DT'),
 ('decision', 'NN')]

El unigram tomó la palabra "object" como sustantivo erroneamente.
El HMM lo tageo correcamente como verbo.

### “Juvenile Court to Try Shooting Defendant”

#### Lo tageo a mano para remover ambiguedad.

Juvenile: Adjetivo

Court: Sustantivo

Try: Verbo

Shooting: Verbo

Defendant: Verbo

In [46]:
title = "Juvenile Court to Try Shooting Defendant"
title_tok = word_tokenize(title)
hmm_tagger.tag(title_tok)

[('Juvenile', 'JJ-TL'),
 ('Court', 'NN-TL'),
 ('to', 'IN'),
 ('Try', 'AT'),
 ('Shooting', 'AT'),
 ('Defendant', 'AT')]